# Text Models

> Fill in a module description here


In [ ]:
#| default_exp text.models

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

In [ ]:
#| export
from fastcore.utils import *  # noqa: F403
from transformers import AutoModelForCausalLM
import torch
from torch import nn
from peft import get_peft_model, LoraConfig  # noqa: F403


In [ ]:
#| export
class CharacterLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(CharacterLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)  # Convert input indices to embeddings
        out, hidden = self.lstm(x, hidden)  # Pass through LSTM layers
        out = self.fc(out)  # Apply fully connected layer to LSTM outputs
        return out, hidden

    def init_hidden(self, batch_size, hidden_size, num_layers):
        # Initialize hidden and cell states with zeros
        h0 = torch.zeros(num_layers, batch_size, hidden_size)
        c0 = torch.zeros(num_layers, batch_size, hidden_size)
        return (h0, c0)



In [ ]:
#| hide
# Example configuration:
vocab_size = 100  # Size of the character vocabulary
embed_size = 128  # Size of the embedding vectors
hidden_size = 256  # Number of hidden units in the LSTM
num_layers = 2  # Number of LSTM layers

lstm_model = CharacterLSTM(vocab_size, embed_size, hidden_size, num_layers)
print(lstm_model)

# Initialize hidden states for a batch size of 32:
batch_size = 32
hidden = lstm_model.init_hidden(batch_size, hidden_size, num_layers)


In [ ]:
#| export
class CausalLMModel(nn.Module):
    def __init__(self, model_name_or_path):
        super(CausalLMModel, self).__init__()
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name_or_path,
        )

    def forward(self, input_id):
        output = self.model(input_ids=input_id, return_dict=False)
        return output

In [ ]:
#| export
def delegate(to, *methods):
    def dec(klass):
        def create_delegator(method):
            def delegator(self, *args, **kwargs):
                obj = getattr(self, to)
                m = getattr(obj, method)
                return m(*args, **kwargs)
            return delegator
        for m in methods:
            setattr(klass, m, create_delegator(m))
        return klass
    return dec

In [ ]:
#| export
class CausalLMPEFTModel(torch.nn.Module):
    def __init__(self, cfg):
        super(CausalLMPEFTModel, self).__init__()
        # PEFT configs
        peft_config = LoraConfig( # type: ignore  # noqa: F405
            lora_alpha=cfg.lora_alpha,
            lora_dropout=cfg.lora_dropout,
            target_modules=cfg.target_modules,
            r=cfg.r,
            bias="none",
            task_type="CAUSAL_LM",
        )
        
        base_model = AutoModelForCausalLM.from_pretrained(
            cfg.model,
            device_map="cpu",
            use_cache=False,
            torch_dtype=torch.bfloat16
        )
        self.model = get_peft_model(base_model, peft_config)  # type: ignore # noqa: F405

    def __getattr__(self, name):
        # Bypass __getattr__ when accessing self.model
        self.model = self.__dict__.get("model", None)
        if self.model and hasattr(self.model, name):
            return getattr(self.model, name)
        raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")


    def forward(self, input_id):
        output = self.model(input_ids=input_id, return_dict=False)
        return output

In [ ]:
#| export
def get_hf_model(cfg):

    # remove the hf:// prefix (added as a workaround to distinguish between local and remote models)
    cfg.model.name = cfg.model.name.replace("hf://", "")
    
    model = AutoModelForCausalLM.from_pretrained(cfg.model.name,
                                                 trust_remote_code=True,
                                                 device_map='cpu',
                                                 )
    config = LoraConfig(  # noqa: F405 # type: ignore
                    r= cfg.model.peft.r,
                    target_modules= cfg.model.peft.target_modules,
                    lora_alpha= cfg.model.peft.lora_alpha,
                    lora_dropout= cfg.model.peft.lora_dropout,
                    bias= "none",
                    task_type= "CAUSAL_LM",
                )
        
    model = get_peft_model(model, config)  # type: ignore # noqa: F405

    return model


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export() # type: ignore  # noqa: E702
